In [1]:
from flask import Flask, flash, request, redirect, url_for,render_template
from flask import send_file

import os
from flask_cors import CORS, cross_origin
from cnnClassifier.utils.common import decodeImage
from cnnClassifier.pipeline.hide_prediction import HidePredictionPipeline
from cnnClassifier.pipeline.reveal_prediction import RevealPredictionPipeline
from PIL import Image
import numpy as np
UPLOAD_FOLDER = 'static'
ALLOWED_EXTENSIONS = {'txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'}
os.putenv('LANG', 'en_US.UTF-8')
os.putenv('LC_ALL', 'en_US.UTF-8')
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
app = Flask(__name__)
CORS(app)

class ClientApp:
    def __init__(self):
        self.secret = "static\images\secret.jpg"
        self.cover = "static\images\cover.jpg"
        self.coverout = "static\images\Coverout.jpg"
        self.toreveal = r"static\images\toreveal.jpg"
        self.secretout = r"static\images\secretout.jpg"
        self.hideclassifier = HidePredictionPipeline(self.secret, self.cover)
        self.revealclassifier = RevealPredictionPipeline()

clApp = ClientApp()

In [6]:
%pwd

'c:\\Users\\08you\\Desktop\\My own 3 hour project\\Securing-Medical-Data-Transfer-Using-Deep-Steganography-and-Cryptography-MLflow-DVC'

In [5]:
import os 
os.chdir("../")
os.chdir("../")

In [7]:
clApp.revealclassifier.predict(clApp.toreveal)

AttributeError: 'JpegImageFile' object has no attribute 'shape'

In [12]:
from tensorflow.keras.preprocessing import image

img =image.load_img(clApp.toreveal, target_size = (224,224))

In [18]:
import numpy as np 
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from cnnClassifier.utils.common import custom_loss_1,custom_loss_2,gray_to_rgb,normalize_batch,denormalize_batch
import os
import tensorflow  as tf
reveal= tf.keras.models.load_model(
    "artifacts/split_model/reveal.h5",custom_objects={'custom_loss_2': custom_loss_2,'custom_loss_1': custom_loss_1}
    , compile=False
)
if len(img.split()) != 3:
    img = gray_to_rgb(img)
secretout=reveal.predict(np.reshape(img,(1,224,224,3)))

1/1 [==============================] - 1s 1s/step
